# Generate Dynamics CUDA code (local output)

This notebook generates CUDA-ready CasADi code directly into `src/generated/` using the new `cuda` CodeGenerator option.


In [1]:
import sys, ctypes

# Force local CasADi package
sys.path.insert(0, "/tmp/casadi-local")

# Ensure the local libcasadi is visible to the loader
ctypes.CDLL("/tmp/casadi-build/lib/libcasadi.so", mode=ctypes.RTLD_GLOBAL)

import casadi as ca
print("CasADi from:", ca.__file__)

CasADi from: /tmp/casadi-local/casadi/__init__.py


In [2]:
import casadi as ca
from pathlib import Path

dynamics_eval = ca.Function.load("dynamics_eval.casadi")

In [3]:
base_dir = Path.cwd().parent
codegen_folder = base_dir / "src" / "generated"
codegen_folder.mkdir(exist_ok=True, parents=True)

fname = dynamics_eval.name()
cg = ca.CodeGenerator("dynamics_blue", {
    "with_header": True,
    "casadi_real": "float",
    "cpp": False,
    "cuda": True,
    "cuda_kernels": {
        fname: {
            "batch_inputs": [2],
        }
    },
})

cg.add(dynamics_eval)
out_path = cg.generate(str(codegen_folder) + "/")
out_path

'/home/mr-robot/sci_ws/casadi-on-gpu/src/generated/dynamics_blue.cu'

In [4]:
(codegen_folder / "dynamics_blue.cu").exists(), (codegen_folder / "dynamics_blue.cuh").exists()

(True, True)